<a id=“title_ID”></a>
# JWST Pipeline Validation Testing Notebook: Spec2, srctype, MIRI
Instruments Affected: MIRI

## Introduction and summary of test being run

This notebook processes an image through calwebb_spec2 and examines the output of the flat_field step. The steps are as follow:

1) Set up data path and directory and image file names.

2) Set the appropriate header keywords to test.

2) Run data through the source_type step in calwebb_spec2.

3) Compare input to output SRCTYPE keyword

## Documentation
The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

##Data used

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public?

Author: Mike Engesser

## Create Temporary Directory

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

<TemporaryDirectory '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmp_211z0pk'>


In [2]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

CRDS cache location: /grp/crds/cache


## Set Up Import Statements

In [3]:
from astropy.io import fits, ascii
import pytest
import numpy as np
import numpy.ma as ma

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_spec2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os
import sys

In [4]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

## Print pipeline version number 

In [5]:
jwst.__version__ 

'1.3.2'

## Read in data from Artifactory

In [6]:
file = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits')

## Define functions for validation and running pipeline

In [7]:
print('[*]', file=open('stpipe-log.cfg', "w"))
print('handler = file:pipeline.log', file=open('stpipe-log.cfg', "a"))
print('level = INFO', file=open('stpipe-log.cfg', "a"))

In [8]:
def run_spec2(im):
    """Creates and runs an instance of the JWST spec2 pipeline. """
    
    #only assign_wcs and srctype are run
    spec2 = calwebb_spec2.Spec2Pipeline()
    spec2.bkg_subtract.skip = True
    spec2.assign_wcs.skip = False
    spec2.imprint_subtract.skip = True
    spec2.msa_flagging.skip = True
    spec2.extract_2d.skip = True
    spec2.wavecorr.skip = True
    spec2.srctype.skip = False
    spec2.straylight.skip = True
    spec2.fringe.skip = True
    spec2.pathloss.skip = True
    spec2.barshadow.skip = True
    spec2.photom.skip = True
    spec2.resample_spec.skip = True
    spec2.cube_build.skip = True
    spec2.extract_1d.skip = True
    spec2.flat_field.skip = True
    spec2.save_results = True
        
    spec2.run(im)

In [9]:
def test_src_type(input_file, im1):
    """Passes data to spec2 pipeline and records the source type keyword of the output"""
    
    #run through spec2
    run_spec2(im1)
    
    #get the output file name
    cal_file = input_file.replace('rate.fits', 'cal.fits')
    
    #create an image model of the output and return the source type keyword
    im2 = ImageModel(cal_file)
    
    # Get the header source type value as well
    hdu=fits.open(cal_file)
    
    return im2.meta.target.source_type,hdu[1].header['SRCTYPE']

## Create an image model and test source type keywords

This test passes if none of the assertions return an error

In [10]:
im = ImageModel(file)

In [11]:
# Test the default case where no external information which should return EXTENDED
im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-01-29 15:44:19,576 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-01-29 15:44:19,578 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-01-29 15:44:19,581 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-01-29 15:44:19,582 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-01-29 15:44:19,584 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-01-29 15:44:26,039 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done


2022-01-29 15:44:26,040 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-01-29 15:44:26,041 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-01-29 15:44:27,019 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-01-29 15:44:27,020 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [12]:
# Test the case where APT specified 'POINT'
im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

im.meta.target.source_type_apt = 'POINT'
    
src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2022-01-29 15:44:28,355 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-01-29 15:44:28,357 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-01-29 15:44:28,359 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-01-29 15:44:28,361 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-01-29 15:44:28,362 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-01-29 15:44:34,052 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done


2022-01-29 15:44:34,053 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-01-29 15:44:34,054 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-01-29 15:44:35,011 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-01-29 15:44:35,012 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [13]:
# Test the case where APT specified 'EXTENDED'
im.meta.target.source_type_apt = 'EXTENDED'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-01-29 15:44:36,261 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-01-29 15:44:36,263 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-01-29 15:44:36,266 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-01-29 15:44:36,269 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-01-29 15:44:36,270 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-01-29 15:44:41,778 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done


2022-01-29 15:44:41,779 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-01-29 15:44:41,780 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-01-29 15:44:42,728 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-01-29 15:44:42,729 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [14]:
# If the background target keyword is set, it should always returns EXTENDED
# regardless of what APT said

im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = True

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-01-29 15:44:43,971 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-01-29 15:44:43,974 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-01-29 15:44:43,976 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-01-29 15:44:43,978 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-01-29 15:44:43,979 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-01-29 15:44:49,449 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done


2022-01-29 15:44:49,450 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-01-29 15:44:49,450 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-01-29 15:44:50,408 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-01-29 15:44:50,409 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [15]:
# If no information from APT, but dither keywords suggest a point source it should
# return POINT

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'POINT-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2022-01-29 15:44:51,657 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-01-29 15:44:51,659 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-01-29 15:44:51,661 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-01-29 15:44:51,662 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-01-29 15:44:51,663 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-01-29 15:44:58,287 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done


2022-01-29 15:44:58,288 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-01-29 15:44:58,289 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-01-29 15:44:59,530 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-01-29 15:44:59,531 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [16]:
# If no information from APT, but dither keywords suggest an extended source it should
# return EXTENDED

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'EXTENDED-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-01-29 15:45:01,171 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-01-29 15:45:01,173 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-01-29 15:45:01,175 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-01-29 15:45:01,176 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-01-29 15:45:01,178 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-01-29 15:45:09,375 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done


2022-01-29 15:45:09,376 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-01-29 15:45:09,377 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-01-29 15:45:10,471 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-01-29 15:45:10,473 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [17]:
# If the observation is a TSO, return POINT

im.meta.observation.bkgdtarg = False
im.meta.visit.tsovisit = True
im.meta.exposure.nints = 3

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2022-01-29 15:45:11,742 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-01-29 15:45:11,745 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-01-29 15:45:11,749 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-01-29 15:45:11,751 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-01-29 15:45:11,753 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-01-29 15:45:17,216 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done


2022-01-29 15:45:17,217 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-01-29 15:45:17,218 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-01-29 15:45:18,188 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-01-29 15:45:18,189 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [18]:
print('No problems found.')

No problems found.
